In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from konlpy.tag import Okt
from multiprocessing.dummy import Pool as ThreadPool
import re
import csv
import numpy as np

In [ ]:
df = pd.read_excel("plutnews.xlsx")
df = df.fillna('')
title = df["title"].tolist()
content = df["content"].tolist()

docs = title + content

In [ ]:
JVM_PATH = "/Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home/bin/java"
okt = Okt(jvmpath=JVM_PATH)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', ',']
X_train = []

def clean_sentence(sentence):
  temp_X = []
  temp_X = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', sentence)
  temp_X = re.sub(r'[0-9]', "", temp_X)
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  temp_X = " ".join(temp_X)
  X_train.append(temp_X)
  
  
pool = ThreadPool(10)

for _ in tqdm(pool.imap_unordered(clean_sentence, docs), total=len(docs)):
    pass

pool.close() 
pool.join()

In [ ]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(X_train)

tfidf = tfidf_vectorizer_vectors.todense()
tfidf[tfidf == 0] = np.nan
means = np.nanmean(tfidf, axis=0)
means = dict(zip(tfidf_vectorizer.get_feature_names(), means.tolist()[0]))

tfidf = tfidf_vectorizer_vectors.todense()
ordered = np.argsort(tfidf*-1)
words = tfidf_vectorizer.get_feature_names()

result = {}

# [t for range(len(words)) in tqdm(enumerate(X_train)) for t in range(len(words))]

for i, doc in tqdm(enumerate(X_train)):
    for t in range(len(words)):
        result[words[ordered[i,t]]] = means[words[ordered[i,t]]]

with open('plus-tf-idf.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in result.items():
       writer.writerow([key, value])

# open
# with open('dict.csv') as csv_file:
#     reader = csv.reader(csv_file)
#     mydict = dict(reader)